  ### CASE STUDY HEALTHCARE DATA ANALYSIS (PART 1) 

#### PART 1: Identify providers that overcharge for certain procedures or regions where procedures are too expensive

In [1]:
import pandas as pd
import numpy as np

In [2]:
Medicare_Charge_In = pd.read_csv('Medicare_Charge_Inpatient_DRG100_DRG_Summary_by_DRG_FY2011.csv')
Medicare_Charge_Out = pd.read_csv('Medicare_Charge_Outpatient_APC30_Summary_by_APC_CY2011.csv')
Medicare_Provider_Charge_In = pd.read_csv('Medicare_Provider_Charge_Inpatient_DRG100_FY2011.csv')
Medicare_Provider_Charge_Out = pd.read_csv('Medicare_Provider_Charge_Outpatient_APC30_CY2011_v2.csv')
Patient_history = pd.read_csv('Patient_history_samp.csv')
Review_patient = pd.read_csv('Review_patient_history_samp.csv')
Rreview_transaction = pd.read_csv('Rreview_transaction_coo.csv')
Transaction= pd.read_csv('Transaction_coo.csv')

    The Data Analysis will be based on financial summary data from 2011 that were made availible by the centers of  Medicare and Medicaid Services
    
    Medicare_Provider_Charge_Inpatient_DRG100_FY2011.csv
    Medicare_Provider_Charge_Outpatient_APC30_CY2011_v2.csv

In [3]:
Medicare_Provider_Charge_In.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163065 entries, 0 to 163064
Data columns (total 12 columns):
DRG Definition                                163065 non-null object
Provider Id                                   163065 non-null int64
Provider Name                                 163065 non-null object
Provider Street Address                       163065 non-null object
Provider City                                 163065 non-null object
Provider State                                163065 non-null object
Provider Zip Code                             163065 non-null int64
Hospital Referral Region (HRR) Description    163065 non-null object
Total Discharges                              163065 non-null int64
Average Covered Charges                       163065 non-null float64
Average Total Payments                        163065 non-null float64
Average Medicare Payments                     163065 non-null float64
dtypes: float64(3), int64(3), object(6)
memory usage: 14.9+ M

In [4]:
Medicare_Provider_Charge_Out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43372 entries, 0 to 43371
Data columns (total 11 columns):
APC                                           43372 non-null object
Provider Id                                   43372 non-null int64
Provider Name                                 43372 non-null object
Provider Street Address                       43372 non-null object
Provider City                                 43372 non-null object
Provider State                                43372 non-null object
Provider Zip Code                             43372 non-null int64
Hospital Referral Region (HRR) Description    43372 non-null object
Outpatient Services                           43372 non-null int64
Average  Estimated Submitted Charges          43372 non-null float64
Average Total Payments                        43372 non-null float64
dtypes: float64(2), int64(3), object(6)
memory usage: 3.6+ MB


   ### PART1A - Highest Cost Variation

    The Highest cost variation can be calculated by dividing the standard deviation by the mean.
    
    
    The 3 Highest cost variation  can be seen in Outpatients for following APCS-
    
    0604 - Level 1 Hospital Clinic Visits
    0698 - Level II Eye Tests & Treatments
    0019 - Level I Excision/ Biopsy

In [5]:
# Finding Standard deviation and mean for InpatientDRG
Medicare_in = pd.concat(
[Medicare_Provider_Charge_In[['DRG Definition','Average Covered Charges']].groupby(['DRG Definition']).agg('std').add_suffix('_std'),
 Medicare_Provider_Charge_In[['DRG Definition','Average Covered Charges']].groupby(['DRG Definition']).agg('mean').add_suffix('_avg')]
 , axis =1)

In [6]:
#Coefficient of Variance, the standard deviation divided by the mean
Medicare_in['Variation'] = Medicare_in.apply(lambda x: x['Average Covered Charges_std']/x['Average Covered Charges_avg'],axis =1)

In [7]:
Medicare_in['Variation'].sort_values(ascending=False).head(3)

DRG Definition
885 - PSYCHOSES                                                              0.717796
897 - ALCOHOL/DRUG ABUSE OR DEPENDENCE W/O REHABILITATION THERAPY W/O MCC    0.633967
917 - POISONING & TOXIC EFFECTS OF DRUGS W MCC                               0.613150
Name: Variation, dtype: float64

In [12]:
# Finding Standard deviation and mean for OutpatientDRG
Medicare_out = pd.concat([Medicare_Provider_Charge_Out[["APC","Average  Estimated Submitted Charges"]].groupby(['APC']).agg('std').add_suffix('_std'),
Medicare_Provider_Charge_Out[["APC","Average  Estimated Submitted Charges"]].groupby(['APC']).agg('mean').add_suffix('_avg')],
axis =1)

In [13]:
#Coefficient of Variance, the standard deviation divided by the mean
Medicare_out['Variation'] = Medicare_out.apply(lambda x: x['Average  Estimated Submitted Charges_std']/x['Average  Estimated Submitted Charges_avg'],axis =1)

In [14]:
Medicare_out['Variation'].sort_values(ascending=False).head(3)

APC
0604 - Level 1 Hospital Clinic Visits     1.385150
0698 - Level II Eye Tests & Treatments    1.310444
0019 - Level I Excision/ Biopsy           1.229897
Name: Variation, dtype: float64

 ### PART1B - Highest Cost Claims by provider
    
    The 3 Highest cost claims can be seen for follwing Providers-
    
    BAYONNE HOSPITAL CENTER       
    CROZER CHESTER MEDICAL CENTER 
    STANFORD HOSPITAL             
    

#### For InPatient

In [15]:
Medicare_Provider_Charge_In.head(2)

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region (HRR) Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,91,32963.07692,5777.241758,4763.736264
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,AL - Birmingham,14,15131.85714,5787.571429,4976.714286


In [16]:
#Finding the max of Average Covered Charges for InpatientDRG
Medicare_in1 = pd.DataFrame(Medicare_Provider_Charge_In[['DRG Definition', 'Average Covered Charges']].groupby(['DRG Definition']).agg('max').add_suffix('_max')).reset_index()

In [17]:
 Medicare_in1.head(3)

,DRG Definition,Average Covered Charges_max
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,146892.0000
1,057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/...,148128.4375
2,064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,234912.7000


In [18]:
#Merging the max value with InpatientDRG dataset
Medicare_in_merg = Medicare_Provider_Charge_In.merge(Medicare_in1,on='DRG Definition', how='left')

In [19]:
Medicare_in_merg.to_csv('Medicare_in_merg.csv')

In [20]:
#getting number of providers charge max value for procedure
Medicare_in_merg.loc[Medicare_in_merg['Average Covered Charges_max']  == Medicare_in_merg['Average Covered Charges'],['Provider Name','Average Covered Charges']].groupby('Provider Name').count().add_suffix('_count').reset_index().sort_values(by=['Average Covered Charges_count'],ascending=False).head(3)

,Provider Name,Average Covered Charges_count
0,BAYONNE HOSPITAL CENTER,23
5,CROZER CHESTER MEDICAL CENTER,11
26,STANFORD HOSPITAL,10


In [21]:
#getting number of times a region has charge max value for procedure
Medicare_in_merg.loc[Medicare_in_merg['Average Covered Charges_max']  == Medicare_in_merg['Average Covered Charges'],['Hospital Referral Region (HRR) Description','Average Covered Charges']].groupby('Hospital Referral Region (HRR) Description').count().add_suffix('_count').reset_index().sort_values(by=['Average Covered Charges_count'],ascending=False).head(3)

,Hospital Referral Region (HRR) Description,Average Covered Charges_count
14,NJ - Newark,25
16,PA - Philadelphia,23
7,CA - San Mateo County,13


#### FOR OutpatientAPC

In [22]:
Medicare_Provider_Charge_Out.columns

Index(['APC', 'Provider Id', 'Provider Name', 'Provider Street Address',
       'Provider City', 'Provider State', 'Provider Zip Code',
       'Hospital Referral Region (HRR) Description', 'Outpatient Services',
       'Average  Estimated Submitted Charges', 'Average Total Payments'],
      dtype='object')

In [23]:
#Finding the max of Average Covered Charges for OutPatientAPC
Medicare_out1 = pd.DataFrame(Medicare_Provider_Charge_Out[['APC', 'Average  Estimated Submitted Charges']].groupby(['APC']).agg('max').add_suffix('_max')).reset_index()

In [24]:
 Medicare_out1.head(2)

,APC,Average Estimated Submitted Charges_max
0,0012 - Level I Debridement & Destruction,841.9
1,0013 - Level II Debridement & Destruction,5231.0


In [25]:
#Merging the max value with InpatientDRG dataset
Medicare_out_merg = Medicare_Provider_Charge_Out.merge(Medicare_out1,on='APC', how='left')

In [26]:
Medicare_out_merg.loc[Medicare_out_merg['Average  Estimated Submitted Charges_max']  == Medicare_out_merg['Average  Estimated Submitted Charges'],['Provider Name','Average  Estimated Submitted Charges']].groupby('Provider Name').count().add_suffix('_count').reset_index().sort_values(by=['Average  Estimated Submitted Charges_count'],ascending=False).head(3)

,Provider Name,Average Estimated Submitted Charges_count
3,COLUMBIA HOSPITAL,2
8,FORT WALTON BEACH MEDICAL CENTER,2
1,CAPITAL HEALTH MEDICAL CENTER - HOPEWELL,2


In [27]:
Medicare_out_merg.loc[Medicare_out_merg['Average  Estimated Submitted Charges_max']  == Medicare_out_merg['Average  Estimated Submitted Charges'],['Hospital Referral Region (HRR) Description','Average  Estimated Submitted Charges']].groupby('Hospital Referral Region (HRR) Description').count().add_suffix('_count').reset_index().sort_values(by=['Average  Estimated Submitted Charges_count'],ascending=False).head(3)

,Hospital Referral Region (HRR) Description,Average Estimated Submitted Charges_count
17,PA - Philadelphia,5
1,CA - Los Angeles,3
10,MA - Boston,2


  ###  PART1C - Highest Cost Claims by Region

    Combining the counts of Inpatient and Outpatients Following are the 3 Highest cost claims by Region -
    
   
    Hospital Referral Region (HRR) Description

1. CA - Contra Costa County 36.0
2. CA - San Mateo County 24.0
3. CA - Santa Cruz 11.0

#### FOR InpatientDRG

In [28]:
#Finding the mean of Average Covered Charges for InpatientDRG by DRG Defination and Region
Medicare_in_rg = pd.DataFrame(Medicare_Provider_Charge_In[['DRG Definition', 'Hospital Referral Region (HRR) Description',  'Average Covered Charges']].groupby(['DRG Definition', 'Hospital Referral Region (HRR) Description']).agg('mean').add_suffix('_avg')).reset_index()

In [29]:
Medicare_in_rg.head(5)

,DRG Definition,Hospital Referral Region (HRR) Description,Average Covered Charges_avg
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AK - Anchorage,34805.130430
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Birmingham,40237.361538
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Dothan,36285.182905
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Huntsville,18878.955554
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Mobile,22802.270118


In [33]:
Medicare_in_max=Medicare_in_rg[['DRG Definition', 'Average Covered Charges_avg']].groupby(['DRG Definition']).agg('max').add_suffix('_max').reset_index()

In [34]:
Medicare_in_max.head(2)

,DRG Definition,Average Covered Charges_avg_max
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,105929.4706
1,057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/...,148128.4375


In [35]:
#Merging The mean and max - will give side by side comparison of region having highest claims
Medicare_in_merg = Medicare_in_rg.merge(Medicare_in_max,on='DRG Definition', how='left')

In [36]:
Medicare_in_merg.head(3)

,DRG Definition,Hospital Referral Region (HRR) Description,Average Covered Charges_avg,Average Covered Charges_avg_max
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AK - Anchorage,34805.130430,105929.4706
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Birmingham,40237.361538,105929.4706
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Dothan,36285.182905,105929.4706


In [86]:
temp1=Medicare_in_merg.loc[Medicare_in_merg['Average Covered Charges_avg']==Medicare_in_merg['Average Covered Charges_avg_max'],['Hospital Referral Region (HRR) Description','Average Covered Charges_avg']].groupby(['Hospital Referral Region (HRR) Description']).count().add_suffix('_count').reset_index()
temp1.head()
temp1

,Hospital Referral Region (HRR) Description,Average Covered Charges_avg_count
0,CA - Alameda County,1
1,CA - Contra Costa County,36
2,CA - Modesto,5
3,CA - Napa,2
4,CA - San Francisco,1
5,CA - San Jose,9
6,CA - San Luis Obispo,6
7,CA - San Mateo County,23
8,CA - Santa Cruz,9
9,CA - Santa Rosa,2


#### FOR OutpatientAPC

In [56]:
Medicare_Provider_Charge_Out.columns

Index(['APC', 'Provider Id', 'Provider Name', 'Provider Street Address',
       'Provider City', 'Provider State', 'Provider Zip Code',
       'Hospital Referral Region (HRR) Description', 'Outpatient Services',
       'Average  Estimated Submitted Charges', 'Average Total Payments'],
      dtype='object')

In [58]:
#Finding the mean of Average Covered Charges for Outpatient by APC and Region
Medicare_out_rg = pd.DataFrame(Medicare_Provider_Charge_Out[['APC', 'Hospital Referral Region (HRR) Description', 'Average  Estimated Submitted Charges']].groupby(['APC', 'Hospital Referral Region (HRR) Description']).agg('mean').add_suffix('_avg')).reset_index()

In [59]:
Medicare_out_rg.head()

,APC,Hospital Referral Region (HRR) Description,Average Estimated Submitted Charges_avg
0,0012 - Level I Debridement & Destruction,AK - Anchorage,149.589749
1,0012 - Level I Debridement & Destruction,AL - Birmingham,78.086957
2,0012 - Level I Debridement & Destruction,AR - Fort Smith,80.102439
3,0012 - Level I Debridement & Destruction,AR - Jonesboro,16.993333
4,0012 - Level I Debridement & Destruction,AR - Little Rock,96.968863


In [61]:
#Finding the max for Outpatient
Medicare_out_max = pd.DataFrame(Medicare_out_rg[['APC', 'Average  Estimated Submitted Charges_avg']].groupby(['APC']).agg('max').add_suffix('_max')).reset_index()

In [62]:
Medicare_out_max.head()

,APC,Average Estimated Submitted Charges_avg_max
0,0012 - Level I Debridement & Destruction,438.307045
1,0013 - Level II Debridement & Destruction,1042.707451
2,0015 - Level III Debridement & Destruction,1478.536545
3,0019 - Level I Excision/ Biopsy,15946.325655
4,0020 - Level II Excision/ Biopsy,15784.975000


In [64]:
#Merging The mean and max - will give side by side comparison of region having highest claims
Medicare_out_merg = Medicare_out_rg.merge(Medicare_out_max,on='APC', how='left')

In [73]:
Medicare_out_merg.head(3)

,APC,Hospital Referral Region (HRR) Description,Average Estimated Submitted Charges_avg,Average Estimated Submitted Charges_avg_max
0,0012 - Level I Debridement & Destruction,AK - Anchorage,149.589749,438.307045
1,0012 - Level I Debridement & Destruction,AL - Birmingham,78.086957,438.307045
2,0012 - Level I Debridement & Destruction,AR - Fort Smith,80.102439,438.307045


In [75]:
Medicare_out_merg.columns

Index(['APC', 'Hospital Referral Region (HRR) Description',
       'Average  Estimated Submitted Charges_avg',
       'Average  Estimated Submitted Charges_avg_max'],
      dtype='object')

In [92]:
temp2=Medicare_out_merg.loc[Medicare_out_merg['Average  Estimated Submitted Charges_avg']==Medicare_out_merg['Average  Estimated Submitted Charges_avg_max'],['Average  Estimated Submitted Charges_avg','Hospital Referral Region (HRR) Description']].groupby(['Hospital Referral Region (HRR) Description']).count().add_suffix('_count').reset_index()
temp2.head()

,Hospital Referral Region (HRR) Description,Average Estimated Submitted Charges_avg_count
0,AL - Dothan,1
1,AL - Huntsville,1
2,CA - Los Angeles,2
3,CA - Modesto,1
4,CA - Redding,1


In [ ]:
temp3['Hospital Referral Region (HRR) Description'].sort_values(by=['sum'],ascending=False).head(3)


In [105]:
temp3=pd.merge(temp1,temp2,how='outer').fillna(0)
temp3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 0 to 30
Data columns (total 3 columns):
Hospital Referral Region (HRR) Description        31 non-null object
Average Covered Charges_avg_count                 31 non-null float64
Average  Estimated Submitted Charges_avg_count    31 non-null float64
dtypes: float64(2), object(1)
memory usage: 992.0+ bytes


In [115]:
# combining both in patient and outpaitent  highest claims by region
temp3['sum']=temp3['Average Covered Charges_avg_count']+temp3['Average  Estimated Submitted Charges_avg_count']

temp3.sort_values(by=['sum'],ascending=False)[['Hospital Referral Region (HRR) Description','sum']].head(3)

,Hospital Referral Region (HRR) Description,sum
1,CA - Contra Costa County,36.0
7,CA - San Mateo County,24.0
8,CA - Santa Cruz,11.0


###  PART1D - Largest claim difference for the largest number of procedures by providers

    We see largest claim difference for highest number of procedures for InpatientDGR for the follwing providers -
    
    BAYONNE HOSPITAL CENTER, NJ          29
    CROZER CHESTER MEDICAL CENTER, PA    12
    HAHNEMANN UNIVERSITY HOSPITAL, PA     8

#### InpatientDRG

    Considering -
    Average Covered Charges as Claims
    Average Total Payments as Reimbursement

In [116]:
Medicare_Provider_Charge_In.head(3)

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region (HRR) Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,91,32963.07692,5777.241758,4763.736264
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,AL - Birmingham,14,15131.85714,5787.571429,4976.714286
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,AL - Birmingham,24,37560.37500,5434.958333,4453.791667


In [118]:
Medicare_in_drg = Medicare_Provider_Charge_In[['DRG Definition', 'Provider Name','Average Total Payments', 'Average Covered Charges' ]]

In [119]:
#Difference of claim and reimbursement
Medicare_in_drg['CLAIM_DIFF'] = Medicare_in_drg.apply(lambda x: x['Average Covered Charges'] - x['Average Total Payments'],axis =1)

C:\Users\arnav\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [121]:
Medicare_in_drg.head(2)

,DRG Definition,Provider Name,Average Total Payments,Average Covered Charges,CLAIM_DIFF
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,SOUTHEAST ALABAMA MEDICAL CENTER,5777.241758,32963.07692,27185.835162
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,MARSHALL MEDICAL CENTER SOUTH,5787.571429,15131.85714,9344.285711


In [122]:
#Finding the mean of Average Covered Charges for InpatientDRG by DRG Defination and Provider
Medicare_in_ag = pd.DataFrame(Medicare_in_drg[['DRG Definition', 'Provider Name', 'CLAIM_DIFF']].groupby(['DRG Definition', 'Provider Name']).agg('mean').add_suffix('_avg')).reset_index()

In [123]:
Medicare_in_ag.head(3)

,DRG Definition,Provider Name,CLAIM_DIFF_avg
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,ABBOTT NORTHWESTERN HOSPITAL,27026.166671
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,ABILENE REGIONAL MEDICAL CENTER,29293.857146
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,ABINGTON MEMORIAL HOSPITAL,33207.235299


In [128]:
#Finding the max for InpatientDRG DRG Definition
Medicare_in_max = pd.DataFrame(Medicare_in_ag[['DRG Definition', 'CLAIM_DIFF_avg']].groupby(['DRG Definition']).agg('max').add_suffix('_max'))

In [129]:
Medicare_in_max.head()

,CLAIM_DIFF_avg_max
DRG Definition,
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,135377.61538
057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/O MCC,122609.00000
064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W MCC,214346.92103
065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W CC,144842.08333
066 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W/O CC/MCC,109535.14290


In [130]:
#Merging The mean and max - will give side by side comparison of providers having highest claim difference
Medicare_in_merg = Medicare_in_ag.merge(Medicare_in_max,on='DRG Definition', how='left').reset_index()

In [131]:
Medicare_in_merg.head()

,index,DRG Definition,Provider Name,CLAIM_DIFF_avg,CLAIM_DIFF_avg_max
0,0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,ABBOTT NORTHWESTERN HOSPITAL,27026.166671,135377.61538
1,1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,ABILENE REGIONAL MEDICAL CENTER,29293.857146,135377.61538
2,2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,ABINGTON MEMORIAL HOSPITAL,33207.235299,135377.61538
3,3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,ADENA REGIONAL MEDICAL CENTER,16247.499998,135377.61538
4,4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,ADVENTIST LA GRANGE MEMORIAL HOSPITAL,30415.357141,135377.61538


In [132]:
temp5=Medicare_in_merg.loc[Medicare_in_merg['CLAIM_DIFF_avg']>=Medicare_in_merg['CLAIM_DIFF_avg_max'],]

In [137]:
##getting number of times a provider has max claim diff for max procudures
temp5[['Provider Name','CLAIM_DIFF_avg']].groupby('Provider Name').count().add_suffix('_count').reset_index().sort_values(by=['CLAIM_DIFF_avg_count'],ascending=False).head(3)

,Provider Name,CLAIM_DIFF_avg_count
0,BAYONNE HOSPITAL CENTER,29
4,CROZER CHESTER MEDICAL CENTER,12
8,HAHNEMANN UNIVERSITY HOSPITAL,8


#### OutPatientsAPC

    Considering -
    Average Estimated Submitted Charges as Claims
    Average Total Payments as Reimbursement

In [138]:
Medicare_Provider_Charge_Out.head(3)

,APC,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region (HRR) Description,Outpatient Services,Average Estimated Submitted Charges,Average Total Payments
0,0012 - Level I Debridement & Destruction,10029,EAST ALABAMA MEDICAL CENTER AND SNF,2000 PEPPERELL PARKWAY,OPELIKA,AL,36801,AL - Birmingham,23,78.086957,21.910435
1,0012 - Level I Debridement & Destruction,20024,CENTRAL PENINSULA GENERAL HOSPITAL,250 HOSPITAL PLACE,SOLDOTNA,AK,99669,AK - Anchorage,994,149.589749,36.623853
2,0012 - Level I Debridement & Destruction,30064,"UNIVERSITY OF ARIZONA MEDICAL CTR-UNIVERSIT, THE",1501 NORTH CAMPBELL AVENUE,TUCSON,AZ,85724,AZ - Tucson,1765,50.135411,14.541841


In [139]:
Medicare_Provider_Charge_Out.columns

Index(['APC', 'Provider Id', 'Provider Name', 'Provider Street Address',
       'Provider City', 'Provider State', 'Provider Zip Code',
       'Hospital Referral Region (HRR) Description', 'Outpatient Services',
       'Average  Estimated Submitted Charges', 'Average Total Payments'],
      dtype='object')

In [140]:
Medicare_out_drg = Medicare_Provider_Charge_Out[['APC', 'Provider Name', 'Average  Estimated Submitted Charges', 'Average Total Payments' ]]

In [141]:
#Difference of claim and reimbursement
Medicare_out_drg['CLAIM_DIFF'] = Medicare_out_drg.apply(lambda x: x['Average  Estimated Submitted Charges'] - x['Average Total Payments'],axis =1)

C:\Users\arnav\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [142]:
Medicare_out_drg.head(3)

,APC,Provider Name,Average Estimated Submitted Charges,Average Total Payments,CLAIM_DIFF
0,0012 - Level I Debridement & Destruction,EAST ALABAMA MEDICAL CENTER AND SNF,78.086957,21.910435,56.176522
1,0012 - Level I Debridement & Destruction,CENTRAL PENINSULA GENERAL HOSPITAL,149.589749,36.623853,112.965895
2,0012 - Level I Debridement & Destruction,"UNIVERSITY OF ARIZONA MEDICAL CTR-UNIVERSIT, THE",50.135411,14.541841,35.593569


In [144]:
#Finding the mean of Average Covered Charges for InpatientDRG by DRG Defination and Provider
Medicare_out_ag = pd.DataFrame(Medicare_out_drg[['APC', 'Provider Name', 'CLAIM_DIFF']].groupby(['APC', 'Provider Name']).agg('mean').add_suffix('_avg')).reset_index()

In [145]:
Medicare_out_ag.head(5)

,APC,Provider Name,CLAIM_DIFF_avg
0,0012 - Level I Debridement & Destruction,ALTON MEMORIAL HOSPITAL,91.856818
1,0012 - Level I Debridement & Destruction,ALTRU HOSPITAL,7.259495
2,0012 - Level I Debridement & Destruction,ARNOT OGDEN MEDICAL CENTER,98.893846
3,0012 - Level I Debridement & Destruction,"AROOSTOOK MEDICAL CENTER,THE",16.975013
4,0012 - Level I Debridement & Destruction,AVERA MCKENNAN HOSPITAL & UNIVERSITY HEALTH CE...,22.306715


In [146]:
#Finding the max for InpatientDRG DRG Definition
Medicare_out_max = pd.DataFrame(Medicare_out_ag[['APC', 'CLAIM_DIFF_avg']].groupby(['APC']).agg('max').add_suffix('_max')).reset_index()

In [147]:
Medicare_out_max.head()

,APC,CLAIM_DIFF_avg_max
0,0012 - Level I Debridement & Destruction,827.44000
1,0013 - Level II Debridement & Destruction,5175.40000
2,0015 - Level III Debridement & Destruction,2980.73963
3,0019 - Level I Excision/ Biopsy,26073.80500
4,0020 - Level II Excision/ Biopsy,18362.75882


In [149]:
#Merging The mean and max - will give side by side comparison of region having highest claim difference
Medicare_out_merg = Medicare_out_ag.merge(Medicare_out_max,on='APC', how='left')

In [150]:
Medicare_out_merg.head()

,APC,Provider Name,CLAIM_DIFF_avg,CLAIM_DIFF_avg_max
0,0012 - Level I Debridement & Destruction,ALTON MEMORIAL HOSPITAL,91.856818,827.44
1,0012 - Level I Debridement & Destruction,ALTRU HOSPITAL,7.259495,827.44
2,0012 - Level I Debridement & Destruction,ARNOT OGDEN MEDICAL CENTER,98.893846,827.44
3,0012 - Level I Debridement & Destruction,"AROOSTOOK MEDICAL CENTER,THE",16.975013,827.44
4,0012 - Level I Debridement & Destruction,AVERA MCKENNAN HOSPITAL & UNIVERSITY HEALTH CE...,22.306715,827.44


In [151]:
temp6=Medicare_out_merg.loc[Medicare_out_merg['CLAIM_DIFF_avg']>=Medicare_out_merg['CLAIM_DIFF_avg_max'],]

In [152]:
##getting number of times a provider has max claim diff for max procudures
temp6[['Provider Name','CLAIM_DIFF_avg']].groupby('Provider Name').count().add_suffix('_count').reset_index().sort_values(by=['CLAIM_DIFF_avg_count'],ascending=False).head(3)

,Provider Name,CLAIM_DIFF_avg_count
3,COLUMBIA HOSPITAL,2
8,FORT WALTON BEACH MEDICAL CENTER,2
1,CAPITAL HEALTH MEDICAL CENTER - HOPEWELL,2
